In [13]:
import os

import scipy.ndimage as nd

import matplotlib.pyplot as plt


import astropy.units as u

from sunpy.net import Fido
from sunpy.net import attrs as a
from sunpy.coordinates import frames

from sunkit_image.coalignment import mapsequence_coalign_by_match_template as mc_coalign
from sunkit_image.coalignment import calculate_match_template_shift as mc_shift

import numpy as np

import cmasher as cmr

from identification_utils import *

from datetime import datetime, timedelta
from astropy.io import fits

In [14]:
###Parameters to load data (see sunpy tutorial)

#mail access
jsoc_email = "stucki@ieec.cat"

#data names
Ic_serie ='hmi.Ic_noLimbDark_720s'
M_serie = 'hmi.M_45s'

#directory to retrieve the files into
path = '/home/sophie-stucki/sunsim/SDO_data/'       #SDO fits
maps_path = '/home/sophie-stucki/sunsim/maps_active/'  #classification maps

#False if the data are already download
download_from_sdo = True

#Delete the SDO files after
delete = False

#Plot the SDO maps and the resulting classification
plot = False #TODO test it

# Set the date range you want to loop over
start_date = datetime.strptime('2019-06-01', '%Y-%m-%d')
end_date = datetime.strptime('2019-09-01', '%Y-%m-%d')  

step_size = 1   #in days

# Downsampling
out_shape = (1024, 1024)

#Noise threshold (from Sen & al. 2023)
noise_thresh = 8

#Contrast threshold
I_th = 0.89

In [15]:
cont_map_list = []
los_map_list = []
t_list = []
days_list = []


# Loop day by day
current_date = start_date
while current_date < end_date:
    # Time range for this day
    start_time = current_date.replace(hour=0, minute=0, second=0)
    end_time = start_time + timedelta(seconds=30)

    # Format if needed
    start_str = start_time.strftime('%Y-%m-%dT%H:%M:%S')
    end_str = end_time.strftime('%Y-%m-%dT%H:%M:%S')

    print(f"Processing {start_str} to {end_str}")

    # Put your SunPy/Fido data access or map logic here

    ###Load the data
    date_str = current_date.strftime('%Y%m%d')
    nbr_day = (current_date - start_date).days

    if download_from_sdo:
        try:
            cont_sequence, los_sequence = load_data(start_time, end_time, jsoc_email, Ic_serie, M_serie, path=path)

            cont_map = cont_sequence.maps[0]
            los_map = los_sequence.maps[0]
            t_list.append(date_str)
            days_list.append(nbr_day)

            cont_map_list.append(cont_map)
            los_map_list.append(los_map)
        except:
            pass

    else:      

        cont_filename = f"{path}hmi.ic_nolimbdark_720s.{date_str}_000000_TAI.3.continuum.fits"
        los_filename = f"{path}hmi.m_45s.{date_str}_000045_TAI.2.magnetogram.fits"

        if os.path.exists(cont_filename) and os.path.exists(los_filename):
            cont_fits = fits.open(cont_filename)
            cont_map = sunpy.map.Map(cont_fits[1].data,cont_fits[1].header, sequence=False, allow_errors=True)

            los_fits = fits.open(los_filename)
            los_map = sunpy.map.Map(los_fits[1].data, los_fits[1].header, sequence=True, allow_errors=True)
        
            t_list.append(date_str)
            days_list.append(nbr_day)

            cont_map_list.append(cont_map)
            los_map_list.append(los_map)
       
    if plot:
        two_graphs_plot(cont_map, los_map)

    if delete:
        if os.path.exists(f"{path}hmi.ic_nolimbdark_720s.{date_str}_000000_TAI.3.continuum.fits"):
            os.remove(f"{path}hmi.ic_nolimbdark_720s.{date_str}_000000_TAI.3.continuum.fits")
            print(f"Deleting {path}hmi.ic_nolimbdark_720s.{date_str}_000000_TAI.3.continuum.fits")
        else:
            print(f"Error: the file {path}hmi.ic_nolimbdark_720s.{date_str}_000000_TAI.3.continuum.fits doesn't exist") 

        if os.path.exists(f"{path}hmi.m_45s.{date_str}_000045_TAI.2.magnetogram.fits"):
            os.remove(f"{path}hmi.m_45s.{date_str}_000045_TAI.2.magnetogram.fits")
            print(f"Deleting {path}hmi.m_45s.{date_str}_000045_TAI.2.magnetogram.fits")
        else:
            print(f"Error: the file {path}hmi.m_45s.{date_str}_000045_TAI.2.magnetogram.fits doesn't exist") 

    # Move to the next day
    current_date += timedelta(days=int(step_size))

np.savetxt(maps_path+'days_list.txt', days_list)

Processing 2019-06-01T00:00:00 to 2019-06-01T00:00:30


2025-09-12 15:53:35 - drms - INFO: Export request pending. [id=JSOC_20250912_002561, status=2]
2025-09-12 15:53:35 - drms - INFO: Waiting for 0 seconds...
2025-09-12 15:53:35 - drms - INFO: Export request pending. [id=JSOC_20250912_002561, status=1]
2025-09-12 15:53:35 - drms - INFO: Waiting for 5 seconds...
2025-09-12 15:53:41 - drms - INFO: Export request pending. [id=JSOC_20250912_002561, status=1]
2025-09-12 15:53:41 - drms - INFO: Waiting for 5 seconds...
2025-09-12 15:53:47 - drms - INFO: Export request pending. [id=JSOC_20250912_002561, status=1]
2025-09-12 15:53:47 - drms - INFO: Waiting for 5 seconds...
2025-09-12 15:53:53 - sunpy - INFO: 1 URLs found for download. Full request totaling 15MB


INFO: 1 URLs found for download. Full request totaling 15MB [sunpy.net.jsoc.jsoc]


Files Downloaded: 100%|██████████| 1/1 [00:09<00:00,  9.71s/file]
2025-09-12 15:54:14 - drms - INFO: Export request pending. [id=JSOC_20250912_002563, status=2]
2025-09-12 15:54:14 - drms - INFO: Waiting for 0 seconds...
2025-09-12 15:54:14 - drms - INFO: Export request pending. [id=JSOC_20250912_002563, status=1]
2025-09-12 15:54:14 - drms - INFO: Waiting for 5 seconds...
2025-09-12 15:54:20 - drms - INFO: Export request pending. [id=JSOC_20250912_002563, status=1]
2025-09-12 15:54:20 - drms - INFO: Waiting for 5 seconds...
2025-09-12 15:54:25 - drms - INFO: Export request pending. [id=JSOC_20250912_002563, status=1]
2025-09-12 15:54:25 - drms - INFO: Waiting for 5 seconds...
2025-09-12 15:54:31 - drms - INFO: Export request pending. [id=JSOC_20250912_002563, status=1]
2025-09-12 15:54:31 - drms - INFO: Waiting for 5 seconds...
2025-09-12 15:54:37 - drms - INFO: Export request pending. [id=JSOC_20250912_002563, status=1]
2025-09-12 15:54:37 - drms - INFO: Waiting for 5 seconds...
2025

INFO: 1 URLs found for download. Full request totaling 15MB [sunpy.net.jsoc.jsoc]


...

INFO: 1 URLs found for download. Full request totaling 15MB [sunpy.net.jsoc.jsoc]


Files Downloaded: 100%|██████████| 1/1 [00:20<00:00, 20.39s/file]
2025-09-16 12:16:53 - drms - INFO: Export request pending. [id=JSOC_20250916_003216, status=2]
2025-09-16 12:16:53 - drms - INFO: Waiting for 0 seconds...
2025-09-16 12:16:54 - drms - INFO: Export request pending. [id=JSOC_20250916_003216, status=1]
2025-09-16 12:16:54 - drms - INFO: Waiting for 5 seconds...
2025-09-16 12:17:00 - drms - INFO: Export request pending. [id=JSOC_20250916_003216, status=1]
2025-09-16 12:17:00 - drms - INFO: Waiting for 5 seconds...
2025-09-16 12:17:06 - drms - INFO: Export request pending. [id=JSOC_20250916_003216, status=1]
2025-09-16 12:17:06 - drms - INFO: Waiting for 5 seconds...
2025-09-16 12:17:11 - drms - INFO: Export request pending. [id=JSOC_20250916_003216, status=1]
2025-09-16 12:17:11 - drms - INFO: Waiting for 5 seconds...
2025-09-16 12:17:17 - drms - INFO: Export request pending. [id=JSOC_20250916_003216, status=1]
2025-09-16 12:17:17 - drms - INFO: Waiting for 5 seconds...
2025

INFO: 1 URLs found for download. Full request totaling 15MB [sunpy.net.jsoc.jsoc]


Files Downloaded: 100%|██████████| 1/1 [00:25<00:00, 25.20s/file]


FileNotFoundError: [Errno 2] No such file or directory: '/home/sophie-stucki/sunsim/maps_active/days_list.txt'

In [ ]:
#Filling factors
ff_sp = []
ff_fc = []

#Cutoff
min_list = []
max_list = []


for i in range(len(t_list)):

    t = t_list[i]
    cont_map = cont_map_list[i]
    if download_from_sdo:
        los_map = los_map_list[i]
    else:
        los_map = los_map_list[i][0]

    print(f"Processing {t}")

    RAW_size = cont_map.dimensions

    # downsampling
    cont_map = cont_map.resample(out_shape * u.pix)
    los_map = los_map.resample(out_shape * u.pix)   


    # coordinates
    xg, yg = coord_grid(los_map)

    # remove the noise
    los_sequence_updated = sunpy.map.MapSequence(noise_threshold(los_map, threshold=noise_thresh)) 
    # remove the foreshortening effects
    los_sequence_updated = sunpy.map.MapSequence(removing_foreshortening_effect(los_sequence_updated.maps[0],xg,yg))

    # crop
    min_p = int(np.argwhere(cont_map.data[int(out_shape[0]/2), :] >= 0 ).min() - 1)
    max_p = int(np.argwhere(cont_map.data[int(out_shape[0]/2), :] >= 0 ).max() + 1)

    min_list.append(min_p)
    max_list.append(max_p)

    cont_map = sunpy.map.sources.HMIMap(cont_map.data[min_p:max_p, min_p:max_p], cont_map.fits_header)
    los_map = sunpy.map.sources.HMIMap(np.abs(los_sequence_updated.maps[0].data[min_p:max_p, min_p:max_p]), los_sequence_updated.maps[0].fits_header)

    xg = xg[min_p:max_p, min_p:max_p]
    yg = yg[min_p:max_p, min_p:max_p]

    # plot the new sdo images
    if plot:
        two_graphs_plot(cont_map, los_map)

    ### identification following Sen & al. 2023

    active_area = active_area_identification(cont_map, los_map, xg, yg, I_th=I_th)
    spot_area, smooth_spot_area, feature_area, smooth_feature_area = active_area_smoothing(active_area, spot_threshold=0.2, spot_kernel_size=2, feature_threshold=0.2, feature_kernel_size=2) 

    area_th = micro_hemisphere_to_arcsec2(los_sequence_updated.maps[0], 20).value
    identification, plage_nbr = network_identification(feature_area, smooth_feature_area, los_map.scale[0].value, area_th, method='scipy')
    nbr, locs, pxl_area = spot_nbr(spot_area, smooth_spot_area)

    # plot the identification map
    if plot:
        identifiation_plot(identification, spot_area, cont_map, locs, filename=path+'identification_map_downsampling_{}_{}.pdf'.format(int(RAW_size[0].value/out_shape[0]), t))

    ### save the faculae and spot maps for starsim

    facula_map = np.copy(identification)

    facula_map[np.isnan(facula_map)] = 0

    # facula_map = np.zeros(np.shape(identification))
    facula_map[np.isnan(cont_map.data)] = np.nan

    spot_map = np.copy(spot_area)
    # spot_map = np.zeros(np.shape(spot_area))
    
    spot_map[np.isnan(spot_map)] = 0
    spot_map[np.isnan(cont_map.data)] = np.nan

    facula_map = np.flip(facula_map)
    spot_map = np.flip(spot_map)

    ff_sp.append(np.nansum(spot_map))
    ff_fc.append(np.nansum(facula_map))



    np.savetxt(maps_path+'faculae_map_{:.1f}.txt'.format(i), facula_map)
    np.savetxt(maps_path+'spot_map_{:.1f}.txt'.format(i), spot_map)
    np.savetxt(maps_path+'cont_map_{:.1f}.txt'.format(i), np.flip(cont_map.data))
    np.savetxt(maps_path+'los_map_{:.1f}.txt'.format(i), np.flip(los_map.data))

np.savetxt(maps_path+'date.txt', np.int64(t_list))
np.savetxt(maps_path+'ff_sp.txt', ff_sp)
np.savetxt(maps_path+'ff_fc.txt', ff_fc)
np.savetxt(maps_path+'min_p.txt', min_list)
np.savetxt(maps_path+'max_p.txt', max_list)


Processing 20150718
751
Processing 20150719
736
Processing 20150720
750
Processing 20150721
708
Processing 20150722
705
Processing 20150723
668
Processing 20150724
664
Processing 20150725
671
Processing 20150726
683
Processing 20150727
685
Processing 20150728
636
Processing 20150729
689
Processing 20150730
692
Processing 20150731
715
Processing 20150801
708
Processing 20150802
756
Processing 20150803
759
Processing 20150804
783
Processing 20150805
763
Processing 20150806
762
Processing 20150807
785
Processing 20150808
784
Processing 20150809
798
Processing 20150810
747
Processing 20150811
721
Processing 20150812
745
Processing 20150813
762
Processing 20150814
763
Processing 20150815
739
Processing 20150816
726
Processing 20150817
747
Processing 20150818
678
Processing 20150819
657
Processing 20150820
630
Processing 20150821
662
Processing 20150822
651
Processing 20150823
659
Processing 20150824
634
Processing 20150825
694
Processing 20150826
668
Processing 20150827
693
Processing 20150